In [ ]:
there are more missing alues in the society and the balcony columns
May be we could drop the society and do some feature engineering with the balcony column 

It depends on the co-relation among the features, either to drop or perform any treatment

#Encoding process
df['location'].value_counts().head(15)

# Either we can apply a supervised Ml on this size?? , np just replace with 2BHK
df['size']=df['size'].fillna('2 BHK')

df['size']=df['size'].apply(lambda x:x[:2])
df['size']=df['size'].astype('int32')

# replacing by the most recurring attribute
df['bath']=df['bath'].fillna('2')
df['balcony']=df['balcony'].fillna('2')



#converting into integer
df['bath']=df['bath'].astype(float)
df['balcony']=df['balcony'].astype(float)

# In order to have a generic house prediction system , preferrably deleting the society feature!!


x=df[['area_type','location','size','total_sqft','bath','balcony']]
y=df['price']

#df['total_sqft']=df['total_sqft'].astype('int32')
df['total_sqft'].value_counts().tail()
#34.46Sq. Meter
# range 
df['total_sqft']=df['total_sqft'].apply(lambda x:x[:5])


# filter out rows with non-numeric values in total_sqft
df = df[df['total_sqft'].str.isnumeric()]

df['total_sqft']=df['total_sqft'].astype(float)

#train and test spliting the data
from sklearn.model_selection import train_test_split as tt
x_train,x_test,y_train,y_test=tt(x,y,test_size=0.3,random_state=4)

# importing all the Regressor models and try checking the the r2score close to 1
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error

regressors = [
    ['Linear Regression:', LinearRegression()],
    ['Ridge Regression:', Ridge()],
    ['Lasso Regression:', Lasso()],
    ['ElasticNet Regression:', ElasticNet()],
    ['Decision Tree Regressor:', DecisionTreeRegressor()],
    ['Random Forest Regressor:', RandomForestRegressor()],
    ['Gradient Boosting Regressor:', GradientBoostingRegressor()],
    ['AdaBoost Regressor:', AdaBoostRegressor()],
    ['KNeighbors Regressor:', KNeighborsRegressor()],
    ['Support Vector Regressor:', SVR()]
]
score=[]
for i,reg in regressors:
    model=reg
    model.fit(x_train,y_train)
    predictn=model.predict(x_test)
    score.append(r2_score(y_test,predictn))
    
    print(i,r2_score(y_test,predictn))
    
Grad= GradientBoostingRegressor()
G_model=Grad.fit(x_train,y_train)

G_pred=G_model.predict(x_test)

print('The gold price: $'+str(G_pred))



# OHE encoding on these data 
df_f=[i for i in df['area_type'].value_counts().index]

for i in df_f:
    df['AREA_'+i]=np.where(df['area_type']==i,1,0)
df=df.drop('availability',1)
df['location'].isnull().sum()

# replacing the missing location data point with the mode of the feature column i.e.., Whitefield
df['location']=df['location'].fillna('Whitefield')

# better to create a fnt for OHE!!
def OHE(df,var,top):
    for i in top:
        df[var+'_'+i]=np.where(df[var]==i,1,0)
    df=df.drop(var,1)
    
top_15=[i for i in df['location'].value_counts().head(15).index]
OHE(df,'location',top_15)
n_df=df['location'].value_counts().head(15).index